In [4]:
import pandas as pd

from google.cloud import bigquery
#from bertopic import BERTopic
#from colorama import Fore, Style
#from pathlib import Path

In [71]:
#df = pd.read_csv('/home/sebastien/code/CuevasEli/NLP-Systematic-review/raw_data/data.csv')
df = pd.read_csv('/home/sebastien/code/CuevasEli/NLP-Systematic-review/raw_data/europe-central2-a_instance-20230905-153639_NLP-Systematic-review_df_with_topics_100.csv')#, delimiter="\t")

In [72]:
df.head()

,Unnamed: 0,abstract_id,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,0,12925107,To investigate the phamacokinetics and serum b...,519,519_meropenem_piperacillin_mic_imipenemcilastatin,"['meropenem', 'piperacillin', 'mic', 'imipenem...",['Meropenem is commonly used to treat lung inf...,meropenem - piperacillin - mic - imipenemcilas...,0.615971,False
1,1,24641302,"Electrobrasion , like dermabrasion , is a meth...",-1,-1_on_and_blood_by,"['on', 'and', 'blood', 'by', 'the', 'that', 't...","[""Depressive and anxiety symptoms in older adu...",on - and - blood - by - the - that - to - of -...,0.000000,False
2,2,24597620,The primary aim of rheumatoid arthritis ( RA )...,-1,-1_on_and_blood_by,"['on', 'and', 'blood', 'by', 'the', 'that', 't...","[""Depressive and anxiety symptoms in older adu...",on - and - blood - by - the - that - to - of -...,0.000000,False
3,3,22489991,Because the Bonfils fibrescope has a semi-rigi...,1965,1965_trachlight_bent_intubation_lightwand,"['trachlight', 'bent', 'intubation', 'lightwan...",['The Eschmann multiple-use introducer is wide...,trachlight - bent - intubation - lightwand - t...,1.000000,True
4,4,9752963,Canal wall down and intact canal wall tympanom...,552,552_tympanoplasty_tympanic_myringoplasty_airbone,"['tympanoplasty', 'tympanic', 'myringoplasty',...",['To compare the effectiveness of paper-patch ...,tympanoplasty - tympanic - myringoplasty - air...,0.122631,False


In [74]:
def compress(df, **kwargs):
    """
    Reduces the size of the DataFrame by downcasting numerical columns
    """
    input_size = df.memory_usage(index=True).sum()/ 1024**2
    print("old dataframe size: ", round(input_size,2), 'MB')
    
    in_size = df.memory_usage(index=True).sum()

    for t in ["float", "integer"]:
        l_cols = list(df.select_dtypes(include=t))

        for col in l_cols:
            df[col] = pd.to_numeric(df[col], downcast=t)

    out_size = df.memory_usage(index=True).sum()
    ratio = (1 - round(out_size / in_size, 2)) * 100
    
    print("optimized size by {} %".format(round(ratio,2)))
    print("new DataFrame size: ", round(out_size / 1024**2,2), " MB")

    return df

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190654 entries, 0 to 190653
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   abstract_id              190654 non-null  int32  
 1   Document                 190654 non-null  object 
 2   Topic                    190654 non-null  int16  
 3   Name                     190654 non-null  object 
 4   Representation           190654 non-null  object 
 5   Representative_Docs      190654 non-null  object 
 6   Top_n_words              190654 non-null  object 
 7   Probability              190654 non-null  float32
 8   Representative_document  190654 non-null  bool   
dtypes: bool(1), float32(1), int16(1), int32(1), object(5)
memory usage: 9.3+ MB


In [75]:
df_2 = compress(df)
#df_2.drop(columns='Unnamed: 0',inplace=True)
df_2.info()

old dataframe size:  13.27 MB
optimized size by 25.0 %
new DataFrame size:  10.0  MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190654 entries, 0 to 190653
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               190654 non-null  int32  
 1   abstract_id              190654 non-null  int32  
 2   Document                 190654 non-null  object 
 3   Topic                    190654 non-null  int16  
 4   Name                     190654 non-null  object 
 5   Representation           190654 non-null  object 
 6   Representative_Docs      190654 non-null  object 
 7   Top_n_words              190654 non-null  object 
 8   Probability              190654 non-null  float32
 9   Representative_document  190654 non-null  bool   
dtypes: bool(1), float32(1), int16(1), int32(2), object(5)
memory usage: 10.0+ MB


In [2]:
# Variable
GCP_PROJECT_SEBT84='useful-proposal-392710'
GCP_REGION='europe-west1'

# BigQuery
BQ_REGION='EU'
BQ_DATASET='enhancing_systemic_reviews_nlp'
TABLE = 'pubmed'

# Cloud Storage
BUCKET_NAME='enhancing_systemic_reviews_nlp'

In [ ]:
full_table_name = f"{GCP_PROJECT_SEBT84}.{BQ_DATASET}.{TABLE}"

client = bigquery.Client()
query_job = client.query(
        f"""
        SELECT
          *
        FROM {GCP_PROJECT_SEBT84}.{BQ_DATASET}.raw_{TABLE}
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
df = results.to_dataframe()

In [ ]:
df.head()

In [ ]:
df_with_topics = pd.concat([df_small['abstract_id'],topic_model.get_document_info(df_train)],axis=1)

In [ ]:
#path = '/home/sebastien/code/CuevasEli/NLP-Systematic-review
path = '/home/sebastien/code/CuevasEli/NLP-Systematic-review/nlp_systematic_review/model'
loaded_model = BERTopic.load(path)

In [ ]:
loaded_model.get_document_info(df['abstract_text'])

In [76]:
def load_data_bq(df: pd.DataFrame
                 , replace: bool):
    full_table_name = f"{GCP_PROJECT_SEBT84}.{BQ_DATASET}.topics_{TABLE}"

    client = bigquery.Client()

    write_mode = "WRITE_TRUNCATE" if replace else "WRITE_APPEND"
    job_config = bigquery.LoadJobConfig(write_disposition=write_mode)

    job = client.load_table_from_dataframe(df, full_table_name, job_config=job_config)
    result = job.result()

    print(f"✅ Data saved to bigquery, with shape {df.shape}")

In [77]:
load_data_bq(df_2, replace=True)

✅ Data saved to bigquery, with shape (190654, 10)


In [ ]:
def preprocess_data(path_to_csv, frac):
    """provide the path to csv and the frac of data you want to use and it will
    return the preprocessed dataset in pd.DataFrame format"""

    assert isinstance(path_to_csv, str), "path_to_csv should be a string"
    df = pd.read_csv(path_to_csv)
    df = df.sort_values(by=['abstract_id', 'line_number'])
    df['abstract_text'] = df['abstract_text'].astype(str)
    concatenated_abstract = df.groupby('abstract_id')['abstract_text'].apply(' '.join).reset_index()
    df = df.merge(concatenated_abstract, on='abstract_id', how='left')
    data = df[['abstract_id', 'abstract_text_y']].drop_duplicates().rename(columns={'abstract_text_y': 'abstract_text'})
    data = data.sample(frac=frac).reset_index(drop=True)

    return data

In [ ]:
def preprocess_data(df, frac=0.2):
    """provide the path to csv and the frac of data you want to use and it will
    return the preprocessed dataset in pd.DataFrame format"""

    assert isinstance(df, pd.DataFrame), "path_to_csv should be a string"
    
    df = df.sort_values(by=['abstract_id', 'line_number'])
    df['abstract_text'] = df['abstract_text'].astype(str)
    concatenated_abstract = df.groupby('abstract_id')['abstract_text'].apply(' '.join).reset_index()
    df = df.merge(concatenated_abstract, on='abstract_id', how='left')
    data = df[['abstract_id', 'abstract_text_y']].drop_duplicates().rename(columns={'abstract_text_y': 'abstract_text'})
    #data = data.sample(frac=frac).reset_index(drop=True)

    return data

In [ ]:
df_2 = preprocess_data(df)

In [ ]:
load_data_bq(df_2,replace=True)

In [ ]:
def get_data_from_bq(table):
    full_table_name = f"{GCP_PROJECT_SEBT84}.{BQ_DATASET}.concat_{TABLE}"

    client = bigquery.Client(project=GCP_PROJECT_SEBT84)
    query = f"""
        SELECT
          count(*)
        FROM {full_table_name}
        """
    query_job = client.query(query)
    
    results = query_job.result()  # Waits for job to complete.
    df = results.to_dataframe()
    count = int(df.values)
    return count

In [ ]:
full_table_name = f"{GCP_PROJECT_SEBT84}.{BQ_DATASET}.concat_{TABLE}"
frac = 5
client = bigquery.Client(project=GCP_PROJECT_SEBT84)
query = f"""
        SELECT
          *
        FROM {full_table_name}
        limit {frac}
        """
query_job = client.query(query)
    
results = query_job.result()  # Waits for job to complete.
df = results.to_dataframe()
df

In [ ]:
size = get_data_from_bq(TABLE)

In [ ]:
round(size * 0.333)

## API v2 - article title

In [19]:
def get_article_title():
    full_table_name = f"{GCP_PROJECT_SEBT84}.{BQ_DATASET}.article_title_2_{TABLE}"

    client = bigquery.Client(project=GCP_PROJECT_SEBT84)
    query = f"""
        SELECT
          article_id
          , article_title
        FROM {full_table_name}
        """
    query_job = client.query(query)
    
    results = query_job.result().to_dataframe()  # Waits for job to complete.

    results[['article_id']] = results[['article_id']].astype(object)
    
    return results

In [23]:
#df = get_article_title()
df.head()

,article_id,article_title
0,18755343,10-year follow-up of a prospective randomized ...
1,17940146,A comprehensive rehabilitation program with qu...
2,17763261,"A one-year randomized double-blind, multicentr..."
3,20151971,A placebo-controlled exploratory study investi...
4,23257370,A prospective randomized trial of thrombectomy...


In [80]:
article_id = 1776

try:
    index_t = df.index[df['article_id'] == article_id].tolist()[0]
    title = df.iloc[index_t]['article_title']
except:
    title = 'No title available for this article.'

title

'No title available for this article.'

In [ ]:
            if article_id in article_titles['article_id'].tolist():
                article_title_index = article_titles.index[article_titles['article_id'] == article_id].tolist()[0]
                article_title = article_titles.iloc[article_title_index]['article_title']
            else:
                article_title = 'No title available for this article.'

In [54]:
index_t = df.index[df['article_id'] == article_id].tolist()[0]

df.iloc[index_t]['article_title']

'A placebo-controlled exploratory study investigating the efficacy and safety of the phosphodiesterase type 5 inhibitor UK-369,003 for the treatment of men with storage lower urinary tract symptoms associated with a clinical diagnosis of overactive bladder.'

In [62]:
article_id in df['article_id'].tolist()

True

In [70]:
from bertopic import *